In [1]:
import sqlite3 as db 
import pandas as pd
conn = db.connect('soccer/database.sqlite')

##
from sklearn.preprocessing import MinMaxScaler

home_adv_factor = 1
# #'attacking_work_rate', 'defensive_work_rate'
# attr_cols = ['overall_rating',
#        'potential', 'crossing', 'finishing', 'heading_accuracy',
#        'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
#        'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
#        'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
#        'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
#        'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
#        'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
#        'gk_reflexes']

attr_cols = ['crossing', 'finishing', 'heading_accuracy','short_passing', 'volleys', 'dribbling', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_positioning','gk_reflexes']

cols = ['Position']


def normalize(df):
    for key in attr_cols:
        #df[key] = pd.to_numeric(df[key])
        scaler = MinMaxScaler()
        df[key] = scaler.fit_transform(df[key].values.reshape(-1,1))
    return df
        

In [2]:
# import unidecode as ud
# import json

# player_pos = json.load(open('player_pos_14.json','r'))
# player_pos_dec = {ud.unidecode(key):value for (key,value) in player_pos.items()}
# print(len(player_pos))
# print(len(player_pos_dec))



In [3]:
norm_team_name = {}
norm_team_name['Man Utd'] = 'Manchester United'
norm_team_name['Newcastle'] = 'Newcastle United'
norm_team_name['Huddersfield'] = 'Huddersfield Town'
norm_team_name['Wolves'] = 'Wolverhampton Wanderers'
norm_team_name['Cardiff'] = 'Cardiff City'
norm_team_name['Leicester'] = 'Leicester City'
norm_team_name['Spurs'] = 'Tottenham Hotspur'
norm_team_name['West Ham'] = 'West Ham United'
norm_team_name['Brighton'] = 'Brighton & Hove Albion'
norm_team_name['Man City'] = 'Manchester City'
norm_team_name['West Ham'] = 'West Ham United'
norm_team_name['Bournemouth'] = 'Bournemouth'

year_data = {'2014': 'statbunker-football-stats/Player Stats 2014-15.csv' , 
             '2015': 'statbunker-football-stats/Player Stats 2015-16.csv' ,
             '2016': 'statbunker-football-stats/Player Stats 2016-17.csv'}


In [4]:
def get_all_players_info(year):
    data = pd.read_csv('fifa-19-player-database/FIFA 19 Player DB.csv', encoding = "ISO-8859-1")
    fixtures = pd.read_csv('fifa-19-player-database/epl-2018-GMTStandardTime.csv', encoding = "ISO-8859-1")
    data = data[data['League'] == 'Premier League']
    data = data.sort_values(by=['Club'])
    return data


In [5]:
def norm_player_names(all_players_info):
    
    for key,value in norm_team_name.items():
        all_players_info.loc[lambda df: df['Home Team'] == key, 'Home Team'] = value
        all_players_info.loc[lambda df: df['Away Team'] == key, 'Away Team'] = value
    
    return all_players_info

In [6]:
def get_full_data(year):
    
    joined  = get_all_players_info(year)
        
    return joined


In [7]:
def create_team(data, name):
    
    full_team = data[data['Club'].str.startswith(name)]
        
#     team_agg = {}
    
#     team_df = {}
    
#     df_start = full_team[full_team['Club'] == 'THIS DOES NOT EXIST']
    
#     pos_sel = {'Defender': 4, 'Midfielder': 4, 'Forward': 2, 'Goalkeeper':1}

# #     if (len(df_start[df_start['Position'] == 'Forward']) < 3 ):
# #         forwards = len(df_start[df_start['Position'] == 'Forward'])
# #         print(forwards)
# #         pos_sel['Forward'] = forwards
# #         pos_sel['Midfielder'] = 6 - forwards

#     for key,value in pos_sel.items():
        
#         grp = full_team[full_team['Position'] == key].sort_values(by = ['overall_rating'], ascending = False).head(value)
#         df_start = df_start.append(grp)
#         avg = grp.mean()
#         team_agg[key] = avg
    
    return full_team

In [8]:
def agg_attrs_all(team):
    
    agg_attrs = {}
    team_atts = team[attr_cols]
    for key in team_atts.keys():
        if key.startswith('gk'):
            agg_attrs[key] = team.sort_values(by = [key], ascending = False)[key].head(2).mean()
        else:
            agg_attrs[key] = team.sort_values(by = [key], ascending = False)[key].head(4).mean()
        
    agg_attrs_df = pd.DataFrame([agg_attrs]).astype(float)
    return agg_attrs_df


def agg_attributes(team):
    
    team.loc[lambda df: df['Position'] == 'Goalkeeper', 'Position'] = 'Defender'
    
    g = team.groupby(['Position'] , as_index=False).mean()
    return g.transpose()

In [9]:
def get_fixtures(year):
    
    fixtures = pd.read_csv('fifa-19-player-database/epl-2018-GMTStandardTime.csv', encoding = "ISO-8859-1")
    fixtures = norm_player_names(fixtures)
    
    return fixtures
    


In [10]:
def combine_metrics_4(i,row,home_attrs,away_attrs,team_form):
   
#     def update_form(home_team_name, away_team_name, result):
#         if result == 1:
#             team_form[home_team_name] += 2
#             team_form[away_team_name] -= 2
#         elif result == -1:
#             team_form[home_team_name] -= 3
#             team_form[away_team_name] += 3
#         else:
#             team_form[home_team_name] -= 1
#             team_form[away_team_name] += 1
        
    norm_attrs = {}
    
    for key in home_attrs.keys():
        norm_attrs['h_'+key] = float(home_attrs[key])
        norm_attrs['a_'+key] = float(away_attrs[key])
    
    norm_attrs['Home_Team'] = row['Home Team']
    norm_attrs['Away_Team'] = row['Away Team']
    norm_attrs['Game_Week'] = (i//10) + 1
    norm_attrs['Home_Form'] = 60 if norm_attrs['Game_Week'] == 1 else 0
    norm_attrs['Away_Form'] = 60 if norm_attrs['Game_Week'] == 1 else 0
                                            
    norm_df = pd.DataFrame(norm_attrs, index = [str(i)])
    
    return norm_df 

In [11]:
team_state = {}


In [12]:
def get_all_team_data(year):
    
    data = {}

    joined_data = get_full_data(year)

    f = get_fixtures(year)
    teams = f['Home Team'].unique() #get all unique teams

    for team in teams:
        team_data = create_team(joined_data, team)
        team_attr = agg_attrs_all(team_data)
        data[team] = (team_data, team_attr)
    return data


In [13]:
def construct_training_data(years):
    
    full_res_df = pd.DataFrame()
    
    for year in years:
        
        team_store = get_all_team_data(year)
        i = 0
        res_df = pd.DataFrame()
        f = get_fixtures(year)
    
        team_form = {name:60 for name in f['Home Team'].unique()}
        for key,row in f.iterrows():
            home = row['Home Team']
            away = row['Away Team']

            home_attrs = team_store[home][1]
            away_attrs = team_store[away][1]

            res_df = res_df.append(combine_metrics_4(i,row,home_attrs,away_attrs,team_form))

            i = i + 1
        
        full_res_df = full_res_df.append(res_df)

    return full_res_df

In [14]:
tr_data = construct_training_data(['2018'])
tr_data

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  import sys


,h_acceleration,a_acceleration,h_aggression,a_aggression,h_agility,a_agility,h_balance,a_balance,h_ball_control,a_ball_control,...,a_strength,h_vision,a_vision,h_volleys,a_volleys,Home_Team,Away_Team,Game_Week,Home_Form,Away_Form
0,87.50,86.25,90.75,86.75,88.25,84.50,88.75,80.50,88.50,80.00,...,87.00,85.25,78.25,80.75,76.25,Manchester United,Leicester City,1,60,60
1,85.50,85.00,82.75,89.50,84.75,85.25,84.25,86.25,78.25,88.50,...,90.50,77.50,85.00,75.25,80.00,Newcastle United,Tottenham Hotspur,1,60,60
2,86.50,86.25,78.75,83.25,84.25,84.00,90.00,83.25,78.25,76.00,...,86.00,76.75,74.25,70.75,69.00,Bournemouth,Cardiff City,1,60,60
3,83.25,85.25,84.25,86.75,81.75,87.00,85.25,87.00,80.50,81.75,...,86.75,78.25,76.25,77.25,69.25,Fulham,Crystal Palace,1,60,60
4,80.75,89.25,83.50,87.50,76.00,89.50,78.75,88.75,78.50,88.25,...,81.75,75.00,88.75,68.75,80.75,Huddersfield Town,Chelsea,1,60,60
5,83.25,83.50,86.00,84.50,82.50,86.25,82.25,84.25,82.25,80.25,...,81.50,79.25,79.50,73.75,72.25,Watford,Brighton & Hove Albion,1,60,60
6,90.25,87.75,81.25,83.25,84.00,84.50,83.25,84.50,81.75,82.50,...,87.50,77.25,79.50,74.00,74.75,Wolverhampton Wanderers,Everton,1,60,60
7,90.25,89.75,85.00,85.50,90.75,84.25,90.00,86.25,88.00,85.25,...,87.75,84.25,80.00,77.00,76.25,Liverpool,West Ham United,1,60,60
8,83.00,81.50,82.75,83.75,83.75,79.25,86.00,80.75,80.50,79.25,...,82.50,76.25,78.25,72.25,71.00,Southampton,Burnley,1,60,60
9,90.00,91.75,89.75,89.50,84.50,93.50,84.75,92.00,85.50,91.50,...,84.75,83.75,90.00,82.00,83.50,Arsenal,Manchester City,1,60,60


In [15]:
tr_data.to_csv('Full_Data_feats_ha.csv')

In [16]:
def generate_final_table(fixtures, results):
    team_points = {}

    for i in range(10):
        team_points[fixtures.iloc[i]['Home Team']] = 0
        team_points[fixtures.iloc[i]['Away Team']] = 0

    for i in range(380):
        result = results[i]
        fixture = fixtures.iloc[i]
        if(result == 1):
            team_points[fixture['Home Team']] += 3
        elif(result == 2):
            team_points[fixture['Home Team']] += 1
            team_points[fixture['Away Team']] += 1
        elif(result == 3):
            team_points[fixture['Away Team']] += 3

    sorted_team_points = sorted(team_points.items(), key=lambda kv: kv[1])
    sorted_team_points.reverse()
    return dict(sorted_team_points)

In [19]:
import random

results = []
fixtures = get_fixtures('2019')
for x in range(380):
  results.append(random.randint(1,3))

final_table = pd.DataFrame.from_dict(generate_final_table(fixtures, results), orient = 'index', columns = ['Points'])
final_table

,Points
Manchester City,63
Bournemouth,62
Burnley,59
Wolverhampton Wanderers,57
Watford,56
Chelsea,56
Everton,54
Brighton & Hove Albion,53
Leicester City,52
West Ham United,50
